In [3]:
import os
import itertools
import torch 
import json
import numpy as np 
import random 
from tqdm import tqdm 


# 用来生成负样本的字典
def get_negative_dict(file):
    with open(file, 'r') as f:
        new_dict = {}
        for attr, attrval_list in json.load(f).items():
            attr_dict = {}
            new_dict[attr] = attr_dict
            for x in attrval_list:
                l = attrval_list.copy()
                l.remove(x)
                x = x.split('=')
                l_noequal = list(map(lambda x: x.split('='), l))
                for k in x:
                    attr_dict[k] = list(itertools.chain.from_iterable(l_noequal))
    return new_dict

attr_dict_file = "original_data/attr_to_attrvals.json"
negative_dict = get_negative_dict(attr_dict_file)

# 用来生成正样本的数据增强字典
def get_positive_dict(file):
    with open(file, 'r') as f:
        new_dict = {}
        for attr, attrval_list in json.load(f).items():
            attr_dict = {}
            new_dict[attr] = attr_dict
            for x in attrval_list:
                x = x.split('=')
                for k in x:
                    attr_dict[k] = x
    return new_dict
attr_dict_file = "original_data/attr_to_attrvals.json"
positive_dict = get_positive_dict(attr_dict_file)

In [5]:
# 对每个属性分离成一个样本
fine_val_path = 'val/fine5000.txt'

attr_val_path = 'val/attribute_val.txt'

val_rets = []

with open(fine_val_path, 'r') as f:
    for i, data in enumerate(tqdm(f)):
        data = json.loads(data)
        if data['key_attr']:
            for query, attr in data['key_attr'].items():
                new_data = {}
                new_data['img_name'] = data['img_name']
                new_data['title'] = data['title']
                new_data['key_attr'] = {}
                new_data['match'] = {}
                new_data['match']['图文'] = 1
                new_data['feature'] = data['feature']
                if random.random() > 0.5:
                    new_data['key_attr'][query] = random.sample(negative_dict[query][attr], 1)[0]
                    new_data['match'][query] = 0
                else:
                    new_data['key_attr'][query] = random.sample(positive_dict[query][attr], 1)[0]
                    new_data['match'][query] = 1
                    
                val_rets.append(json.dumps(new_data, ensure_ascii=False)+'\n')
            
        # if i>500:
        #     break
        # i += 1
        

print(len(val_rets))


with open(attr_val_path, 'w') as f:
    f.writelines(val_rets)

5000it [00:10, 469.84it/s]


10932
